# Prepare CAMELS-spat metadata file

In [3]:
import sys
import pandas as pd
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import python_cs_functions as cs

### Config handling

In [4]:
# Specify where the config file can be found
config_file = '../0_config/config.txt'

In [5]:
# Get the required info from the config file
data_path    = cs.read_from_config(config_file,'data_path')
shps_path    = cs.read_from_config(config_file,'ref_shps_path')
cs_meta_path = cs.read_from_config(config_file,'cs_basin_path')
cs_meta_name = cs.read_from_config(config_file,'cs_meta_name')
ref_can_rhbn_file = cs.read_from_config(config_file,'can_rhbn_meta_url')
ref_can_hydt_file = cs.read_from_config(config_file,'can_hydat_db_url')
ref_usa_topo_file = cs.read_from_config(config_file,'us_camels_topo_url')
ref_usa_name_file = cs.read_from_config(config_file,'us_camels_name_url')

### Load the metadata files

In [6]:
# Define RHBN and HYDAT metadata file location
ref_can_rhbn_name = ref_can_rhbn_file.split('/')[-1].strip().replace('xlsx','csv') # file name
ref_can_rhbn_data = Path(data_path) / shps_path / 'RHBN-CAN' / ref_can_rhbn_name # full path to file
ref_can_hydt_name = ref_can_hydt_file.split('/')[-1].strip().replace('.zip','_RHBN_2020_subset.csv')
ref_can_hydt_data = Path(data_path) / shps_path / 'RHBN-CAN' / ref_can_hydt_name 

In [7]:
# Define CAMELS-US metadata file locations
ref_usa_topo_name = ref_usa_topo_file.split('/')[-1].strip().replace('txt','csv')
ref_usa_topo_meta = Path(data_path) / shps_path / 'CAMELS-US' / ref_usa_topo_name
ref_usa_name_name = ref_usa_name_file.split('/')[-1].strip().replace('txt','csv')
ref_usa_name_meta = Path(data_path) / shps_path / 'CAMELS-US' / ref_usa_name_name

In [8]:
# Load the metadata files
rhbn_data = pd.read_csv(str(ref_can_rhbn_data))
hydt_data = pd.read_csv(str(ref_can_hydt_data))
cmls_topo = pd.read_csv(str(ref_usa_topo_meta))
cmls_name = pd.read_csv(str(ref_usa_name_meta))

In [9]:
# Merge the RHBN and HYDAT files into one
rhbn_meta = rhbn_data.merge(hydt_data, on='STATION_NUMBER')

In [10]:
# Merge the CAMELS files into one
cmls_meta = cmls_name.merge(cmls_topo, on='gauge_id')

### Make data folder

In [30]:
cs_meta_path = Path(data_path) / cs_meta_path
cs_meta_path.mkdir(parents=True, exist_ok=True)

### Make the csv

In [25]:
# Concatenate RHBN and CAMELS-US metadata
# Note: we do this in a loop so we can harmonize the different column names. Not super elegant but it's a one-off thing\
# Note: appending to lists is much quicker than appending to dataframe (https://stackoverflow.com/a/56746204)
data = []
# CANADA
for i,row in rhbn_meta.iterrows(): 
    data.append(['CAN',                                      # Country
                 row['STATION_NUMBER'], row['STATION_NAME'], # Station ID, Station name
                 row['LATITUDE'], row['LONGITUDE'], 'HYDAT', # Station lat, station lon, station source
                 -999, -999, 'n/a',                          # Outlet lat, outlet lon, outlet source (to be filled later)
                 -999, -999,                                 # Mapped lat, lon onto river (to be filled later)
                 -999, -999, 'n/a',                          # Manual outlet lat, manual outlet lon, flag (to be filled later)
                 -999,                                       # CAMELS-spat basin area (to be filled later)
                 'HYDAT gross drainage area', row['DRAINAGE_AREA_GROSS'], # Reference area 1 source, reference area 1 km2
                 'HYDAT effective drainage area', row['DRAINAGE_AREA_EFFECT'], # Reference area 2 source, reference area 2 km2
                 'n/a', 'n/a', -999, -1])                    # Do we have a ref shape?, ref shape source, shape area, overlap

# CAMELS-US
for i,row in cmls_meta.iterrows(): 
    data.append(['USA',                                      # Country
                 f"{row['gauge_id']:08}", row['gauge_name'], # Station ID, Station name // gauges need leading zeroes for IDs <8 numbers for sorting and searching on USGS database purposes
                 row['gauge_lat'],row['gauge_lon'], 'CAMELS-US', # Station lat, station lon, station source
                 -999, -999, 'n/a',                          # Outlet lat, outlet lon, outlet source (to be filled later)
                 -999, -999,                                 # Mapped lat, lon onto river (to be filled later)
                 -999, -999, 'n/a',                          # Manual outlet lat, manual outlet lon, flag (to be filled later)
                 -999,                                       # CAMELS-spat basin area (to be filled later)
                 'GAGES II area', row['area_gages2'],        # Reference area 1 source, reference area 1 km2
                 'Geospatial fabric area', row['area_geospa_fabric'], # Reference area 1 source, reference area 1 km2
                 'n/a', 'n/a', -999, -1])                    # Do we have a ref shape?, ref shape source, shape area, overlap

# Convert to dataframe
cs_meta = pd.DataFrame(data, columns = ['Country',
                                        'Station_id' , 'Station_name',
                                        'Station_lat', 'Station_lon', 'Station_source',
                                        'Outlet_lat' , 'Outlet_lon', 'Outlet_source',
                                        'Mapped_lat' , 'Mapped_lon',
                                        'Manual_lat' , 'Manual_lon', 'Manual_outlet_location',
                                        'Basin_area_km2',
                                        'Ref_area_1_src', 'Ref_area_1_km2',
                                        'Ref_area_2_src', 'Ref_area_2_km2',
                                        'Ref_shape', 'Ref_shape_source', 'Ref_shape_area_km2','Ref_and_new_shape_fractional_overlap'])

In [31]:
# Save the file
cs_meta.to_csv(cs_meta_path/cs_meta_name, encoding='utf-8', index=False)